In [41]:
import pandas as pd
import os
import numpy as np
import re
import datetime

In [42]:
ref_df = pd.read_csv('which_race.csv', header=0)
csv_list = os.listdir('clean_frames')
counter = 34

In [43]:
def distances(x):
    return x.values[:,None]-x.values

In [44]:
# lst = list(range(counter))
# bad_lst =[]

# lst.remove(12)
# bad_lst.append(12)

# lst.remove(14)
# bad_lst.append(14)

# lst.remove(19)
# bad_lst.append(19)

# lst.remove(30)
# bad_lst.append(30)

# lst.remove(31)
# bad_lst.append(31)

In [ ]:
# for race_id in lst:
# for race_id in bad_lst:
# for race_id in [12]:

In [45]:
d2 = {}
d3 = {}
d4 = {}
for race_id in range(counter):
#     print(race_id)
    d2['race_df_'+str(race_id)]=pd.read_csv('clean_frames/race_df_'+str(race_id)+'.csv')
    d3['pit_stop_df_'+str(race_id)]=pd.read_csv('clean_frames/pit_stop_df_'+str(race_id)+'.csv')
    race_df = d2['race_df_'+str(race_id)]
    # dfone = pd.concat(d2.values())
    pit_stop_df = d3['pit_stop_df_'+str(race_id)]

    race_df['TIME'] = race_df['TIME'].apply(lambda x: int(x.split(':')[0])*60 + float(x.split(':')[1]))

    race_df['cumtime']=race_df[['TIME','NO']].groupby('NO').cumsum(skipna=True)

    race_df.drop(columns = ['Unnamed: 0'], inplace=True)

    pit_stop_df.drop(columns = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 6'], inplace=True)

    last_lap = max(race_df['LAP'])
    
    race_df.set_index(['NO','LAP'], inplace=True)
    
    race_df = race_df[~race_df.index.duplicated(keep='first')]

    d_gaps={}
    for i in range(0,last_lap):
        lap = i+1
        gaps_df = pd.DataFrame(distances(race_df['TIME'][race_df.index.get_level_values(1)==lap]),columns=list(race_df.index.get_level_values(0)[race_df.index.get_level_values(1)==lap].values),index = race_df.index.get_level_values(0)[race_df.index.get_level_values(1)==lap].values)

        gaps_df['LAP'] = lap

        gaps_df.reset_index(level=0, inplace=True)

        gaps_df.rename({'index':'NO'},axis='columns',inplace=True)  

    #     gaps_df.set_index(['NO','LAP'], inplace=True)

        d_gaps['gaps_df_lap_'+str(lap)] = gaps_df

    #     gaps_df.drop(columns = ['Unnamed: 0'], inplace=True)


    for i in range(0,last_lap):
        lap = i+1
        gaps_df = d_gaps['gaps_df_lap_'+str(lap)]    
        gaps_df.set_index(['NO','LAP'], inplace=True)
        if lap==1:
            race_df = race_df.merge(gaps_df, on=['NO','LAP'], how="outer")
        else:
            race_df.update(gaps_df, join='left')#, overwrite=True, filter_func=None, raise_conflict=False)

    new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')
    # new_df.drop(columns = ['Unnamed: 0','Unnamed: 0'], inplace=True)

    new_df.set_index(['NO','LAP'], inplace=True)

    # new_df['stop_count'] = np.nan
    new_df['stop_count'] = new_df['STOP']
    ###filling the first lap stop_count0s
    new_df['stop_count'][(new_df.index.get_level_values(1)==1) & (new_df['STOP']!=1)] = 0

    new_df['d_stop'] = new_df['STOP']>0
    new_df['target'] = np.nan

    for j in new_df.index.get_level_values(0).unique():  
        x = pd.concat([new_df['stop_count'][new_df.index.get_level_values(0)==j].unstack(0).ffill().stack(0)],axis=1).swaplevel();
        x.columns = ['stop_count'];
        new_df.update(x,join='left');
        new_df['target'][(new_df.index.get_level_values(0)==j)] = new_df['d_stop'][(new_df.index.get_level_values(0)==j)].shift(-1)
    d4['lap_level_df_'+str(race_id)] = new_df
# new_df['stop_count'][new_df.index.get_level_values(0)==19]

0


/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set o

1


/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set o

2


...

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set o

32


/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set o

33


/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set o

In [53]:
full_df = pd.concat([v for k,v in d4.items()],ignore_index=False)

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [60]:
full_df.shape

list(full_df.columns)

features = full_df[['TIME', 'cumtime', 5,7,6,33,19, 44,55,27,
3,            14,            11,            22,
30,            94,             9,            12,
77,             8,            88,            21,
20,'stop_count',
            'd_stop',            26,            31,
                  47,            18,             2,            36,
                  40,            10,            39,            28]]

target = full_df['target']

In [56]:
full_df.to_csv('full_data.csv')

In [81]:
# df= pd.DataFrame([new_df['d_stop'][(new_df.index.get_level_values(0)==5)],new_df['d_stop'][(new_df.index.get_level_values(0)==5)].shift(-1)])
# new_df['stop_count'][new_df.index.get_level_values(0)==19]

In [484]:
# new_df['STOP'].update(new_df['STOP'].groupby(level=0).ffill())

In [494]:
# new_df.update(new_df.groupby(level=0).ffill())
# df = df.groupby(level='group').bfill()
# df = df.unstack(0).bfill().stack().swaplevel(0, 1).reindex_like(df)
# # Column specific
# df.Value = df.groupby(level=0).Value.bfill()

In [656]:
# for _n in new_df.index.get_level_values(0).unique():
#     print(new_df[new_df.index.get_level_values(0)==_n]['STOP'])

In [302]:
# for x in d_gaps.keys():
#      print(d_gaps[x].shape)
# d_gaps['gaps_df_lap_1'].NO.apply(type)
# d_gaps['gaps_df_lap_1'].LAP.apply(type)

In [ ]:
# how do i cycle through laps without loop? can i vectorize?

# how do i impute nan's for running and RF? choose a time which should not matter based on intuition

# exclude teammate from traffic when feature engineering

# tuesday it is 3 minutes (8-10 quick slides)

# wednesday science fair 
'''there will be a screen on a table
CONTEXT talk about the relevance of the problem 10%
PROCESS general strokes of what the model and why i chose to do it like that 40%+
RESULTS results = metrics (predictive power, importance, inference?) 40%+
NEXT STEPS 10%
 '''



In [91]:
from itertools import compress

In [76]:
lst = [0,1,'a']

In [92]:
fil = [type(e)==int for e in lst]

list(compress(lst, fil))